In [1]:
import weave
import keras
from keras.datasets import cifar10
from keras.utils.np_utils import to_categorical

/Users/Tim/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
model = weave.pyrm_net((32,32,3),3,5,4)

Number of layers 2
number of units 2
number of devices 4
Final Layer Size 1


In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape)
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
print(y_test.shape)

(50000, 3, 32, 32)
(10000, 10)


In [4]:
viewer = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True)
model.fit(x_train[0:100], y_train[0:100], callbacks=[viewer])

Epoch 1/1
100/100 [==============================] - 1s 13ms/step - loss: 13.4465 - acc: 0.1600
